# Amazon photo downloader

In [1]:
REVIEW_PATH = './Sports_and_Outdoors.jsonl'
META_PATH = './meta_Sports_and_Outdoors.jsonl'
SAVE_PATH = './photo_embeddings_2023.ent'
SAVE_RAW_EMB_PATH = './photo_embeddings_2023_raw.jsonl'
SUBSET = 1

## Product filtering

In [2]:
import pandas as pd
import json
import gc

data = []
with open(REVIEW_PATH) as f:
    for line in f:
        entry = json.loads(line)

        entry = {
            'parent_asin': entry['parent_asin']
        }

        data.append(entry)

df = pd.DataFrame(data)
df.head(5)

,parent_asin
0,B0BGFR76CF
1,B00NXQLFQQ
2,B0957WLR63
3,B00IET8S80
4,B01C2SW7XA


In [3]:
item_counts = df.groupby("parent_asin").size().reset_index(name="review_count")
item_counts.head(5)

,parent_asin,review_count
0,0007325614,1
1,0201397544,7
2,030827962X,4
3,0316287229,11
4,0318279622,7


In [4]:
item_counts.sort_values(by=['review_count'], ascending=False, inplace=True)

count = int(item_counts.shape[0] * SUBSET)
item_subset = item_counts.iloc[:count]

item_subset.head(10)

,parent_asin,review_count
542655,B00NWXLQD2,30369
992373,B07BQRWTDJ,23638
1578374,B0C5RBPW2Y,20298
788846,B01L6RE7Z4,16590
1468642,B09MJKJYLQ,15679
1524787,B0B7J8Y581,14565
1578780,B0C5XW2T2N,14478
1466402,B09LW2KHPM,14029
1532183,B0BBFB48YQ,13986
1561750,B0BTNZ41Y7,13856


In [5]:
df = df[df['parent_asin'].isin(item_subset['parent_asin'])]
df.head(5)

,parent_asin
0,B0BGFR76CF
1,B00NXQLFQQ
2,B0957WLR63
3,B00IET8S80
4,B01C2SW7XA


## (item_id, url) preparation

In [6]:
data = []
with open(META_PATH) as f:
    for line in f:
        entry = json.loads(line)

        images_list = entry['images']

        if len(images_list) == 0:
            continue

        entry = {
            'parent_asin': entry['parent_asin'],
            'url': images_list[0].get('thumb', None),
        }

        data.append(entry)

df = pd.DataFrame(data)
df.head(5)

,parent_asin,url
0,B01HDXC8AG,https://m.media-amazon.com/images/I/510tgKWHp2...
1,B07R5BQ4YD,https://m.media-amazon.com/images/I/31tHUCk7ls...
2,B003K8GZ7G,https://m.media-amazon.com/images/I/41RVlcUTMs...
3,B08GC4GBWB,https://m.media-amazon.com/images/I/41bDwvySkD...
4,B07BYV947H,https://m.media-amazon.com/images/I/41FrdMQKGi...


In [7]:
df = df[df['parent_asin'].isin(item_subset['parent_asin'])]
df.head(5)

,parent_asin,url
0,B01HDXC8AG,https://m.media-amazon.com/images/I/510tgKWHp2...
1,B07R5BQ4YD,https://m.media-amazon.com/images/I/31tHUCk7ls...
2,B003K8GZ7G,https://m.media-amazon.com/images/I/41RVlcUTMs...
3,B08GC4GBWB,https://m.media-amazon.com/images/I/41bDwvySkD...
4,B07BYV947H,https://m.media-amazon.com/images/I/41FrdMQKGi...


## Parallel requests with exponential backoff

In [8]:
FILE_SYSTEM_PREFIX = './images/'
WINDOW_SIZE = 100
BASE_DELAY = 1
PATIENCE = 3
CONCURRENCY = 5
USER_AGENTS = [
    # Chrome (Windows/Mac/Linux)
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",

    # Firefox (Windows/Mac/Linux)
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:127.0) Gecko/20100101 Firefox/127.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14.5; rv:127.0) Gecko/20100101 Firefox/127.0",
    "Mozilla/5.0 (X11; Linux i686; rv:127.0) Gecko/20100101 Firefox/127.0",

    # Safari (Mac/iOS)
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 17_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Mobile/15E148 Safari/604.1",

    # Edge (Windows)
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0",

    # Mobile Devices (Android/Samsung)
    "Mozilla/5.0 (Linux; Android 14; SM-S928U) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.147 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 14; Pixel 8 Pro) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.147 Mobile Safari/537.36",

    # Tablets
    "Mozilla/5.0 (iPad; CPU OS 17_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 14; SM-X810) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.6422.147 Safari/537.36",

    # Less Common Browsers
    "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Vivaldi/6.6",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 OPR/102.0.0.0"
]

In [9]:
import asyncio
import aiohttp
from tqdm import tqdm
import random
import uuid
import base64

data = []
async def make_request(item_id, url, session, semaphore):
    global data
    for i in range(PATIENCE):
        try:
            async with semaphore:
                headers = {
                    "User-Agent": random.choice(USER_AGENTS),
                    "Accept-Language": random.choice(["en-US", "en-GB", "en-CA"])
                }
                cookies={
                    "session-id": str(uuid.uuid4()),
                    "ubid-acbca": random.randint(1000000000, 9999999999)
                }

                async with session.get(url, headers=headers, cookies=cookies) as response:
                    if response.status == 200: # OK
                        photo_data = await response.read()

                        data.append({
                            'parent_asin': item_id,
                            'photo_bytes': base64.b64encode(photo_data)
                        })

                    elif response.status in (429, 503): # backoff
                        retry_after = response.headers.get("Retry-After")
                        wait_time = float(retry_after) if retry_after else BASE_DELAY * (2 ** i)
                        await asyncio.sleep(wait_time)

                    elif 500 <= response.status < 600:
                        wait_time = BASE_DELAY * (2 ** i)
                        await asyncio.sleep(wait_time)

                    else: # Something bad happened
                        break

        except (aiohttp.ClientError, asyncio.TimeoutError): # backoff
            wait_time = BASE_DELAY * (2 ** i) * (1 + random.random())
            await asyncio.sleep(wait_time)

    data.append({'parent_asin': item_id, 'photo_bytes': None})


async def handle_batch(batch, semaphore):
    global data
    async with aiohttp.ClientSession() as session:
        tasks = [make_request(item_id, url, session, semaphore) for item_id, url in batch if url]

        await asyncio.gather(*tasks)

        # This is not a bottleneck!
        for photo_entry in data:
            if photo_entry['photo_bytes']:
                with open(f"{FILE_SYSTEM_PREFIX}{photo_entry['parent_asin']}.jpg", 'wb') as o:
                    o.write(photo_entry['photo_bytes'])

        data = []


async def download():
    semaphore = asyncio.Semaphore(CONCURRENCY)

    for i in tqdm(range(0, count, WINDOW_SIZE)):
        batch = df.iloc[i:i+WINDOW_SIZE]

        item_ids = batch['parent_asin'].tolist()
        urls = batch['url'].tolist()

        await handle_batch(zip(item_ids, urls), semaphore)

In [10]:
#await download()

## Photo embedding

In [11]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
from tqdm import tqdm

device = "cuda"
print(f'Using device: {device}')

model = resnet50(weights="IMAGENET1K_V1")
model.classifier = torch.nn.Identity()  # Remove final classification layer
model.eval().to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/piotrs/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 90.8MB/s]


In [12]:
def embed_images(batch):
    tensors = torch.stack([transform(img) for img in batch]).to(device)
    with torch.no_grad():
        emb = model(tensors)
    return emb.cpu().numpy()

In [13]:
import os
seq = []

for index, entry in tqdm(df.iterrows(), total=len(df)):
    if os.path.exists(f"{FILE_SYSTEM_PREFIX}{entry['parent_asin']}.jpg"):
        seq.append(entry['parent_asin'])

100%|██████████| 1586679/1586679 [00:51<00:00, 30823.41it/s]


In [ ]:
data = []
BATCH_SIZE = 16
image_batch = []
item_id_batch = []

for item_id in tqdm(seq):

    image = Image.open(f"{FILE_SYSTEM_PREFIX}{item_id}.jpg").convert("RGB")
    image_batch.append(image)
    item_id_batch.append(item_id)

    if len(image_batch) == BATCH_SIZE:
        embeddings = embed_images(image_batch)

        for i, embedding in enumerate(embeddings):
            data.append({
                'parent_asin': item_id_batch[i],
                'embedding': embedding
            })

        image_batch = []
        item_id_batch = []

if image_batch:
    embeddings = embed_images(image_batch)

    for i in range(len(embeddings)):
        data.append({
            'parent_asin': item_id_batch[i],
            'embedding': embeddings[i]
        })

 91%|█████████▏| 1448480/1585872 [1:13:07<48:15:24,  1.26s/it]

## Save raw embeddings

In [ ]:
ser_data = [
    {
        'parent_asin': entry['parent_asin'],
        'embedding': entry['embedding'].tolist()
    }
    for entry in data
]

with open(SAVE_RAW_EMB_PATH, 'w') as f:
    json.dump(ser_data, f)

## (Incremental) PCA for dimention reduction

###### Or use PCA for better quality, if you have 32GB ram

In [ ]:
from sklearn.decomposition import IncrementalPCA

iPCA = IncrementalPCA(n_components=64)
emb_64 = iPCA.fit_transform(data)
print(emb_64.explained_variance_ratio_.sum())

In [ ]:
iPCA = IncrementalPCA(n_components=128)
emb_128 = iPCA.fit_transform(data)
print(emb_64.explained_variance_ratio_.sum())

## Create .ent file

In [ ]:
df = df[['parent_asin']]
df.rename(columns={'parent_asin': 'ent_id:token'}, inplace=True)

df['ent_emb:float_seq'] = [' '.join(map(str, vec)) for vec in emb_128]
df.head(5)

In [ ]:
df.to_csv(SAVE_PATH, sep='\t', index=False)